These data come from a survey conducted by INSEE, Budget des Familles, which are protected by the Statistical Secrecy Committee. To reproduce the analysis, one must request access to this data.

In [38]:
import pandas as pd
date_analysis = "2023-04"
df_inflation_by_household = pd.read_csv(f'BDF/computed_inflation_by_household_{date_analysis}.csv', index_col="IDENT_MEN")

# Household

In [39]:
df_menage = pd.read_csv("BDF/Csv/MENAGE.csv", sep=";", encoding='latin1')

C:\Users\mdaversa\AppData\Local\Temp\ipykernel_16816\1221685487.py:1: DtypeWarning: Columns (21,22,23,48,49,421) have mixed types. Specify dtype option on import or set low_memory=False.
  df_menage = pd.read_csv("BDF/Csv/MENAGE.csv", sep=";", encoding='latin1')


In [40]:
df_menage = df_menage.set_index("IDENT_MEN")

In [41]:
df_dep_men = pd.read_csv("BDF/Csv/DEPMEN.csv", sep=";", encoding='latin1')

C:\Users\mdaversa\AppData\Local\Temp\ipykernel_16816\3018729231.py:1: DtypeWarning: Columns (354,387,390,395,430,452,502,732,737,766,770,774,778,782,850,1325,1501,1629) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dep_men = pd.read_csv("BDF/Csv/DEPMEN.csv", sep=";", encoding='latin1')


In [42]:
df_dep_men = df_dep_men.set_index("IDENT_MEN")

In [43]:
df_menage = pd.merge(left=df_menage, right=df_dep_men, left_index=True, right_index=True)

In [44]:
variables = ["AGEPR", "TUU", "DNIVIE1", "Stalog", "NENFANTS", "COUPLEPR", "Chaufp"]
df_filtered = df_menage.loc[:, variables]

# Cleaning variables

In [45]:
def clean_variable(df, variable, variable_ref, numerical, labels=[],  bins=[]):
    df[f'{variable}_category'] = pd.cut(x=df[variable], bins=bins, labels=labels, right=True) if numerical else df[variable]
    df = pd.concat([df, pd.get_dummies(data=df[f"{variable}_category"])], axis=1)
    df = df.drop([variable, f"{variable}_category", variable_ref], axis=1)
    return df


Age

In [46]:
df_filtered = clean_variable(df=df_filtered, variable="AGEPR", labels=["Moins de 30 ans", "De 30 à 44 ans", "De 45 à 59 ans", "De 60 à 74 ans", "75 ans et plus"], variable_ref = "De 45 à 59 ans", bins = [0, 30, 45, 60, 75, 102], numerical=True)

Type menage

TUUU

In [47]:
df_filtered = clean_variable(df=df_filtered, variable="TUU", variable_ref = "Ville moyenne", numerical=True, bins=[0,2,4,6,7,9], labels=["Rural", "Petite ville", "Ville moyenne", "Grande ville", "Agglomération parisienne"])

Qintiles niveau de vie

In [48]:
df_filtered = clean_variable(df=df_filtered, variable="DNIVIE1", variable_ref = "Moyen", numerical=True, bins=[0,3,8,11], labels=["Pauvre", "Moyen", "Riche"])

In [49]:
df_filtered = df_filtered.dropna()

In [50]:
df_filtered = clean_variable(df=df_filtered, variable="COUPLEPR", variable_ref = "N'habite pas en couple", numerical=True, bins=[0,1,4], labels=["Habite en couple", "N'habite pas en couple"])

Stalog

In [51]:
df_filtered = clean_variable(df=df_filtered, variable="Stalog", variable_ref = "Locataire", numerical=True, bins=[0,3,6], labels=["Propriétaire", "Locataire"])

In [52]:
df_filtered = clean_variable(df=df_filtered, variable="Chaufp", variable_ref = "Chauffage central ou mixte", numerical=True, bins=[0,4,5,8], labels=["Chauffage central ou mixte", "Chauffage électrique individuel", "Chauffage non-électrique"])

# Regression


In [53]:
df_filtered = pd.merge(df_filtered, df_inflation_by_household, left_index=True, right_index=True)

In [54]:
df_filtered = df_filtered.dropna()


In [55]:
y = df_filtered['inflation']
df_filtered = df_filtered.drop(columns=['inflation'])
X = df_filtered


In [56]:
import statsmodels.api as sm
X = sm.add_constant(X)

In [57]:
model = sm.OLS(y, X)

In [58]:
results = model.fit()


In [59]:
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:              inflation   R-squared:                       0.071
Model:                            OLS   Adj. R-squared:                  0.070
Method:                 Least Squares   F-statistic:                     80.72
Date:                Tue, 13 Jun 2023   Prob (F-statistic):          1.56e-239
Time:                        11:17:21   Log-Likelihood:                -33604.
No. Observations:               15945   AIC:                         6.724e+04
Df Residuals:                   15929   BIC:                         6.736e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     